In [10]:
import os
import sys

from openai import OpenAI

from src.evaluation_utils import parse_response

sys.path.insert(0, '../')

from src.utils import ModelInformation
from src.mlflow_utils import mlflow, download_run_data
import re
import pandas as pd
import numpy as np
import json
# from src.genai_prompts import *
from src.genai_prompts_no_ref import prompt_1_ref_free, prompt_2_ref_free, prompt_3_ref_free, prompt_4_ref_free, prompt_5_ref_free, prompt_6_ref_free
from src.genai_prompts_adaptions import prompt_default, prompt_0, prompt_1, prompt_2, prompt_3
import glob

In [11]:
selected_runs = [  # ModelInformation("wiktionary", "2bba85fab20b4ed6a1a89b0503fd5274", "rep-penality=1.5"),
    ModelInformation("wiktionary", "a8fa79a989fc4a9fb7c876be01ac42d2", "rep-penality=1.5"),
    ModelInformation("distillation", "d0a3f0265f404e48a07b56d7dc9e3f1b", "rep-penality=1.5")
    ]

# prompts = [prompt_mlflow, prompt_default, prompt_1, prompt_2, prompt_3, prompt_4]
# prompts = [prompt_default, prompt_0, prompt_1, prompt_2, prompt_3]
prompts = [prompt_6_ref_free]

def create_request(name, prompt_id, model, prompt_fn, data):
    content = [
        
    ]
    for j, row in enumerate(data.iterrows()):
        row = row[1]
        content.append(
            {"custom_id": f"{name}-{prompt_id}-{j}", "method": "POST", "url": "/v1/chat/completions", "body": {"model": model, "messages": [{"role": "user", "content": prompt_fn(
                row["context_sentence"], row["context_word"], row["gt"], row["prediction"]
            )}],"max_tokens": 1000}}
        )
    return content

In [12]:
selected_runs = tuple(set(selected_runs))
exports = []

for run in selected_runs:
    artifacts = mlflow.artifacts.list_artifacts(run_id=run.name)
    for artifact in artifacts:
        if artifact_name := re.match(r"evaluation_[0-9a-z]*.json", artifact.path):
            artifact_name = artifact_name.group(0)
            evaluation_data = download_run_data(run.name, artifact_name)
            if not evaluation_data:
                raise RuntimeError(f"Could not download evaluation data for run {run}")

            collected_data = pd.DataFrame(data=evaluation_data['data'], columns=evaluation_data['columns'])
            # sample = collected_data.sample(frac=0.1, random_state=42)
            sample = collected_data.sample(frac=0.01, random_state=42)
            # res = sample_to_sw(sample, run)
            # exports.append((run, sample))
            # with open(f"export/export_{run.name}_{run.type}.json", "w+") as file:
            #     json.dump(res, file)
            
            # os.makedirs(f"../openai_batches/{run.name}", exist_ok=True)            
            # for i, prompt in enumerate(prompts):
            #     with open(f"../openai_batches/{run.name}/{i}.json", "w+") as outfile:
            #         json.dump(create_request(run.name, i, "gpt-4o-2024-08-06", prompt, sample), outfile)


In [4]:
len(sample)

342

In [23]:
from tqdm.auto import tqdm

client = OpenAI(api_key=os.environ['OPENAI_API'])

batch_files = sorted(glob.glob("/home/jfeil/MasterThesis/openai_batches/*/*.json"))

for batch_file in tqdm(batch_files):
    responses = []
    with open(batch_file, "r") as f:
        json_data = json.load(f)
    for question in tqdm(json_data, leave=False):
        body = question["body"]
        resp = client.chat.completions.create(model=body['model'],
          messages=body['messages'])
        responses.append(resp.choices[0].message.content)
    output_dir, output_file = os.path.split(batch_file)
    output_dir = os.path.join(output_dir, "output")
    os.makedirs(output_dir, exist_ok=True)
    json.dump(responses, open(f"{output_dir}/{output_file}", "w+"))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/342 [00:00<?, ?it/s]

In [7]:
len(sorted(glob.glob("/home/jfeil/MasterThesis/openai_batches*/*/*.json")))

48

In [8]:
48*342

16416

In [8]:
batch_files

['/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/0.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/1.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/2.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/3.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/4.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/0.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/1.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/2.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/3.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/4.json']

('/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2',
 '0.json')

In [62]:
resp.choices[0].message.content

str

In [54]:
json_data[0]['body']['messages']

[{'role': 'user',
  'content': 'Task:\nYou must return the following fields in your response in two lines, one below the other:\nscore: Your numerical score for the model\'s answer_similarity based on the rubric\njustification: Your reasoning about the model\'s answer_similarity score\n\nYou are an impartial judge. You will be given an input that was sent to a machine\nlearning model, and you will be given an output that the model produced. You\nmay also be given additional information that was used by the model to generate the output.\n\nYour task is to determine a numerical score called answer_similarity based on the input and output.\nA definition of answer_similarity and a grading rubric are provided below.\nYou must use the grading rubric to determine your score. You must also justify your score.\n\nExamples could be included below for reference. Make sure to use them as references and to\nunderstand them before completing the task.\n\nInput:\nSpielschöpfungen Was ist die Definiti

In [38]:
client.files.list()

SyncPage[FileObject](data=[FileObject(id='file-pvFLSPWZHml06266gb97yam4', bytes=2115229, created_at=1725758250, filename='5.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-GhuPwH9oKbm2anWY0pfZqSgJ', bytes=1502365, created_at=1725758243, filename='4.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-fBjSiFjECXkfPx3jJzOvsZXR', bytes=1609411, created_at=1725758240, filename='3.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-TJq4L7h5S8hogFE6bIqGDrhn', bytes=1436235, created_at=1725758238, filename='2.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-TsXFz517mMVMNXEDFYFGn7M3', bytes=2049783, created_at=1725758236, filename='1.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-vcZY0ctd6sUm78WUpCTuSSSE', bytes=1620355, created_at=17257

In [9]:
sorted(glob.glob("/home/jfeil/MasterThesis/openai_batches/*/*.json"))

['/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/0.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/1.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/2.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/3.json',
 '/home/jfeil/MasterThesis/openai_batches/a8fa79a989fc4a9fb7c876be01ac42d2/4.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/0.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/1.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/2.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/3.json',
 '/home/jfeil/MasterThesis/openai_batches/d0a3f0265f404e48a07b56d7dc9e3f1b/4.json']

In [3]:
client.files.list()
for f in client.files.list():
    client.files.delete(f.id)
client.files.list()

NameError: name 'client' is not defined

In [13]:
def parse_response(d):
    d = d.split("\n")
    if "score" in d[0]:
        score = (int(d[0].replace("score:", "").strip()) - 1) / 4.0
    else:
        score = (int(d[0].replace("Punktzahl:", "").strip()) - 1) / 4.0
    
    if "justification" in d[1]:
        justification = d[1].replace("justification:", "").strip()
    else:
        justification = d[1].replace("Begründung:", "").strip()
    return score, justification

In [14]:
from collections import defaultdict
human_eval_dump = pd.read_json("human_eval_dump.json")


human_eval = defaultdict(list)
column_header = []


for a, b in zip(human_eval_dump.columns[1:-1:2], human_eval_dump.columns[2:-1:2]):
    print(a, b)
    a_name = a.split('_')[1]
    b_name = b.split('_')[1]
    name = f"{b_name}-{a_name}"
    column_header.append(name)
    # human_eval[name] = (human_eval_dump[a] / (human_eval_dump[a] + human_eval_dump[b]))
    # human_eval[name] = human_eval[name].fillna(value=-1, inplace=False).to_list()
name = f"{a_name}-{b_name}"
column_header.append(name)

for _, row in human_eval_dump.iterrows():
    vals = []
    for a in range(1, len(row)-1, 2):
        if row.iloc[a] + row.iloc[a+1] > 0:
            v = row.iloc[a+1] / (row.iloc[a] + row.iloc[a+1])
        else:
            v = -1.0
        vals += [v]
        if a == 5:
            if v != -1.0:
                vals += [row.iloc[a] / (row.iloc[a] + row.iloc[a+1])]
            else:
                vals += [-1.0]
    human_eval[row.iloc[0].lower()[:30]] += vals
human_eval = dict(human_eval)

0_GT 0_wiktionary
1_GT 1_distillation
2_wiktionary 2_distillation


In [15]:
n_c = {c: [] for c in column_header}
for a, c in enumerate(column_header):
    for i in range(len(sample)):
        row = sample.iloc[i]
        name = f"{row.title}: {row.context_sentence}"
        n_c[c].append(human_eval[name.lower()[:30]][a])

In [39]:
import pandasgui


output_dfs = {selected_run.name: pd.DataFrame() for selected_run in selected_runs}
name_mapping = {selected_run.name: selected_run.type for selected_run in selected_runs}
for run in selected_runs:
    artifacts = mlflow.artifacts.list_artifacts(run_id=run.name)
    for artifact in artifacts:
        if artifact_name := re.match(r"evaluation_[0-9a-z]*.json", artifact.path):
            artifact_name = artifact_name.group(0)
            evaluation_data = download_run_data(run.name, artifact_name)
            if not evaluation_data:
                raise RuntimeError(f"Could not download evaluation data for run {run}")

            collected_data = pd.DataFrame(data=evaluation_data['data'], columns=evaluation_data['columns'])
            sample = collected_data.sample(frac=0.01, random_state=42)
            # for k, v in n_c.items():
            #     sample[k] = v

            output_dfs[run.name] = sample[['title', 'context_sentence', 'context_word', 'gt', 'prediction', 'meteor_score', 'rouge_L', 'bertscore_f1', 'moverscore_score', 'sentence-embedding_score']]  # , *list(n_c.keys())
            
new_columns = {selected_run.name: {"score": {}, "just": {}} for selected_run in selected_runs}

for file in sorted(glob.glob("/home/jfeil/MasterThesis/openai_batches*/*/output/*.json")):
    with open(file, "r") as f:
        data = json.load(f)
    path_components = file.split("/")
    col = []
    just = []
    for el in data:
        s, j = parse_response(el)
        col.append(s)
        just.append(j)
    new_columns[path_components[5]]["score"][f"score_{path_components[4]}_{path_components[7].replace('.json', '')}"] = col
    new_columns[path_components[5]]["just"][f"just_{path_components[4]}_{path_components[7].replace('.json', '')}"] = just
    
for key in new_columns:
    for col in new_columns[key]["score"]:
        output_dfs[key][col] = new_columns[key]["score"][col]
    for k, v in n_c.items():
        output_dfs[key][k] = v

    output_dfs[key]["_score_avg"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].mean(axis=1)
    output_dfs[key]["_score_std"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].std(axis=1)
    output_dfs[key]["_score_median"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].median(axis=1)
    output_dfs[key]["_score_min"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].min(axis=1)
    output_dfs[key]["_score_max"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].max(axis=1)
    # for col in new_columns[key]["just"]:
    #     output_dfs[key][col] = new_columns[key]["just"][col]

    pandasgui.show(**{name_mapping[key]: output_dfs[key]})


PandasGUI INFO — pandasgui.gui — Opening PandasGUI
/tmp/ipykernel_15660/666936314.py:51: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/666936314.py:51: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/666936314.py:51: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/666936314.py:51: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future versio

In [79]:
import pandasgui

output_dfs = {selected_run.name: pd.DataFrame() for selected_run in selected_runs}

for run in selected_runs:
    artifacts = mlflow.artifacts.list_artifacts(run_id=run.name)
    for artifact in artifacts:
        if artifact_name := re.match(r"evaluation_[0-9a-z]*.json", artifact.path):
            artifact_name = artifact_name.group(0)
            evaluation_data = download_run_data(run.name, artifact_name)
            if not evaluation_data:
                raise RuntimeError(f"Could not download evaluation data for run {run}")

            collected_data = pd.DataFrame(data=evaluation_data['data'], columns=evaluation_data['columns'])
            sample = collected_data.sample(frac=0.01, random_state=42)
    
            output_dfs[run.name] = sample[['title', 'context_sentence', 'context_word', 'gt', 'prediction', 'rouge_L', 'meteor_score', 'bertscore_f1', 'moverscore_score', 'sentence-embedding_score']]
            
new_columns = {selected_run.name: {"score": {}, "just": {}} for selected_run in selected_runs}

for file in sorted(glob.glob("/home/jfeil/MasterThesis/openai_batches*/*/output/*.json")):
    with open(file, "r") as f:
        data = json.load(f)
    path_components = file.split("/")
    col = []
    just = []
    for el in data:
        s, j = parse_response(el)
        col.append(s)
        just.append(j)
    new_columns[path_components[5]]["score"][f"score_{path_components[4]}_{path_components[7].replace('.json', '')}"] = col
    new_columns[path_components[5]]["just"][f"just_{path_components[4]}_{path_components[7].replace('.json', '')}"] = just
    
for key in new_columns:
    for col in new_columns[key]["score"]:
        output_dfs[key][col] = new_columns[key]["score"][col]
    output_dfs[key]["_score_avg"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].mean(axis=1)
    output_dfs[key]["_score_std"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].std(axis=1)
    output_dfs[key]["_score_median"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].median(axis=1)
    output_dfs[key]["_score_min"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].min(axis=1)
    output_dfs[key]["_score_max"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].max(axis=1)
    # for col in new_columns[key]["just"]:
    #     output_dfs[key][col] = new_columns[key]["just"][col]
    for k, v in human_eval.items():
        output_dfs[key][k] = v
    pandasgui.show(**{key: output_dfs[key]})


ValueError: Length of values (4) does not match length of index (342)

In [58]:
human_eval

{'spielschöpfung: "die grafiken ': [1.0, -1.0, 1.0],
 'universitätsbildung: "ich würd': [1.0, 0.8, 1.0],
 'kehrseite: "es bleibt also nur': [-1.0, 1.0, 0.5],
 'mehrmalig: "(...) nur durch ei': [-1.0, 1.0, 0.5],
 'teotihuacan: die archäologisch': [1.0, 0.4, 0.0],
 'auswirkung: "man muss bezweife': [1.0, 1.0, 1.0],
 'tüte: obwohl seine eltern dabe': [1.0, -1.0, 1.0],
 'krankheitssymptom: "häufige kr': [-1.0, -1.0, 0.5],
 'zupflocken: eine bemerkung wut': [0.0, 1.0, 1.0],
 'vorfahrt: wir müssen einen pla': [1.0, 1.0, 1.0],
 '911: ""ich war in der 4. klass': [1.0, 1.0, 1.0],
 'abgasmanipulation: die höhe de': [0.75, 1.0, 0.0],
 'koalitionskrise: "nach kritik,': [1.0, 0.0, 0.0],
 'alex: ""alex, zieh nicht alles': [0.75, -1.0, -1.0],
 'weltraummonster: "und in der e': [1.0, 1.0, 0.5],
 'klassenchef: "wenn man heute d': [1.0, 1.0, 1.0],
 'klaue: er starrte auf die schm': [-1.0, 1.0, 0.6666666666666666],
 'kindesliebe: "er hätte gern me': [0.0, 1.0, 0.0],
 'über den jordan gehen: "auch f': [0.

In [23]:
metrics = [
    "meteor_score", "rouge_L", "bertscore_f1", "moverscore_score", "sentence-embedding_score", "score_openai_batches_ref-based-spaces-2_0", "score_openai_batches_ref-free-0_0",
    "wiktionary-GT", "distillation-GT", "distillation-wiktionary", "wiktionary-distillation"
]

In [27]:
correl = {}
for key in output_dfs:
    correl[name_mapping[key]] = (output_dfs[key][output_dfs[key]["distillation-wiktionary"] != -1][metrics].corr(numeric_only=True))

pandasgui.show(**correl)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
/tmp/ipykernel_15660/599958060.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/599958060.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/599958060.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/599958060.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, i

In [41]:
from scipy.stats import pearsonr, spearmanr

def pearson_corr_pval(df):
    cols = df.columns
    corr_matrix = df.corr("spearman", numeric_only=True)  # Correlation matrix
    pval_matrix = pd.DataFrame(index=cols, columns=cols)  # P-value matrix

    # Calculate pairwise correlations and p-values
    for col1 in cols:
        for col2 in cols:
            if col1 == col2:
                pval_matrix.loc[col1, col2] = 0.0
            else:
                corr, pval = spearmanr(df[col1], df[col2])
                a = ""
                if pval < 0.001:
                    a = f'***'
                elif pval < 0.01:
                    a = f'**'
                elif pval < 0.05:
                    a = f'*'
                pval_matrix.loc[col1, col2] = pval

    return corr_matrix, pval_matrix


correl_sig = {}
for key in output_dfs:
    v, p = pearson_corr_pval(output_dfs[key][metrics])
    correl_sig[name_mapping[key] + "_p"] = p
    correl_sig[name_mapping[key] + "_v"] = v

pandasgui.show(**correl_sig)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
/tmp/ipykernel_15660/82393773.py:33: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/82393773.py:33: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/82393773.py:33: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_15660/82393773.py:33: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, i

In [28]:
output_dfs[key][output_dfs[key]["distillation-wiktionary"] != -1][metrics]

,meteor_score,rouge_L,bertscore_f1,moverscore_score,sentence-embedding_score,score_openai_batches_ref-based-spaces-2_0,score_openai_batches_ref-free-0_0,wiktionary-GT,distillation-GT,distillation-wiktionary,wiktionary-distillation
7104,0.074627,0.166667,0.755442,0.577084,0.495855,0.25,0.25,0.0,-1.0,0.0,1.0
18986,0.409174,0.526316,0.775524,0.644347,0.786072,0.75,0.50,0.0,0.2,0.0,1.0
20011,0.126050,0.111111,0.641628,0.567130,0.492855,0.00,0.25,-1.0,0.0,0.5,0.5
11052,0.000000,0.000000,0.607628,0.515254,0.375760,0.25,0.00,-1.0,0.0,0.5,0.5
29716,0.072464,0.153846,0.702133,0.561744,0.563949,0.25,0.00,0.0,0.6,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
11502,0.120929,0.071429,0.728742,0.559734,0.716578,0.00,0.50,0.0,0.5,0.5,0.5
24673,0.086207,0.222222,0.833634,0.542826,0.436823,0.50,0.25,1.0,0.0,0.0,1.0
11044,0.997685,1.000000,1.000000,0.999853,1.000000,1.00,0.75,1.0,0.0,0.0,1.0
5893,0.647131,0.555556,0.867624,0.644645,0.797369,0.25,0.00,0.0,0.0,0.0,1.0


In [33]:
output_dfs[key][output_dfs[key]["distillation-wiktionary"] != -1].columns[5:]

Index(['meteor_score', 'rouge_L', 'bertscore_f1', 'moverscore_score',
       'sentence-embedding_score', 'wiktionary-GT', 'distillation-GT',
       'distillation-wiktionary', 'wiktionary-distillation',
       'score_openai_batches_ref-based-spaces-2_0',
       'score_openai_batches_ref-based-spaces-2_1',
       'score_openai_batches_ref-based-spaces-2_2',
       'score_openai_batches_ref-based-spaces-2_3',
       'score_openai_batches_ref-based-spaces-2_4',
       'score_openai_batches_ref-based-spaces_0',
       'score_openai_batches_ref-based-spaces_1',
       'score_openai_batches_ref-based-spaces_2',
       'score_openai_batches_ref-based-spaces_3',
       'score_openai_batches_ref-based-spaces_4',
       'score_openai_batches_ref-based_0', 'score_openai_batches_ref-based_1',
       'score_openai_batches_ref-based_2', 'score_openai_batches_ref-based_3',
       'score_openai_batches_ref-based_4', 'score_openai_batches_ref-based_5',
       'score_openai_batches_ref-free-0_0', 'score_

In [81]:
output_dfs[key].columns

Index(['title', 'context_sentence', 'context_word', 'gt', 'prediction',
       'rouge_L', 'meteor_score', 'bertscore_f1', 'moverscore_score',
       'sentence-embedding_score', 'score_openai_batches_0',
       'score_openai_batches_ref-based-spaces-2_0',
       'score_openai_batches_ref-based-spaces-2_1',
       'score_openai_batches_ref-based-spaces-2_2',
       'score_openai_batches_ref-based-spaces-2_3',
       'score_openai_batches_ref-based-spaces-2_4',
       'score_openai_batches_ref-based-spaces_0',
       'score_openai_batches_ref-based-spaces_1',
       'score_openai_batches_ref-based-spaces_2',
       'score_openai_batches_ref-based-spaces_3',
       'score_openai_batches_ref-based-spaces_4',
       'score_openai_batches_ref-based_0', 'score_openai_batches_ref-based_1',
       'score_openai_batches_ref-based_2', 'score_openai_batches_ref-based_3',
       'score_openai_batches_ref-based_4', 'score_openai_batches_ref-based_5',
       'score_openai_batches_ref-free_0', 'score_o

In [42]:
output_dfs[key]["score_std"] = output_dfs[key].loc[:,output_dfs[key].columns.str.contains("^score*")].std(axis=1)

7104     0.469668
18986    1.104802
20011    0.332106
11052    0.931476
29716    0.799816
           ...   
11502    0.966345
24673    0.919559
11044    0.717430
5893     0.696631
10166    0.507300
Length: 342, dtype: float64

In [ ]:
output_dfs[key]

In [22]:
human_eval_dump[a] / (human_eval_dump[a] + human_eval_dump[b])

0      1.0
1      1.0
2      0.5
3      0.5
4      0.0
      ... 
337    0.5
338    1.0
339    1.0
340    1.0
341    1.0
Length: 342, dtype: float64

In [13]:
sample

,title,context_sentence,context_word,gt,prediction,bertscore_precision,bertscore_recall,bertscore_f1,bleurt_score,meteor_score,moverscore_score,rouge_1,rouge_2,rouge_L,rouge_Lsum,sentence-embedding_score
7104,Spielschöpfung,"""Die Grafiken zeigen sowohl tradierte Spiele a...",Spielschöpfungen,kreative Neugestaltung einer Spielform oder ei...,künstlerisches Werk eines Spiels,0.758563,0.752345,0.755442,0.325514,0.074627,0.577084,0.166667,0.000000,0.166667,0.166667,0.495855
18986,Universitätsbildung,"""Ich würde gerne daran festhalten, dass verfüg...",Universitätsbildung,"Bildung, die in einem Universitätsstudium erwo...","Bildung: Schulbildung, die in einer Universitä...",0.805159,0.747993,0.775524,0.598530,0.409174,0.644347,0.526316,0.117647,0.526316,0.526316,0.786072
20011,Kehrseite,"""Es bleibt also nur die Kehrseite des Guten, a...",Kehrseite,"die andere, entgegengesetzte, oftmals negative...","übertragen:: diejenige Seite, die sich von etw...",0.644235,0.639041,0.641628,0.475293,0.126050,0.567130,0.222222,0.000000,0.111111,0.111111,0.492855
11052,mehrmalig,"""(...) nur durch ein mehrmaliges Werfen des sc...",mehrmaliges,"attributiv gebraucht, auch adverbial: mehr als...",mit mehreren vervielfacht,0.640531,0.577940,0.607628,0.240579,0.000000,0.515254,0.000000,0.000000,0.000000,0.000000,0.375760
29716,Teotihuacan,Die archäologische Stätte Teotihuacans gehört ...,Teotihuacans,prähistorische Ruinenstadt im Zentralen Hochla...,Geografie: eine Stadt in Mexiko,0.739982,0.667968,0.702133,0.186324,0.072464,0.561744,0.153846,0.000000,0.153846,0.153846,0.563949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11502,Lehnübersetzung,Ein Beispiel für eine Lehnübersetzung ist das ...,Lehnübersetzung,"Linguistik: ein (auch zusammengesetztes) Wort,...",Linguistik: Übersetzung eines Wortes aus einer...,0.795192,0.672541,0.728742,0.554034,0.120929,0.559734,0.071429,0.000000,0.071429,0.071429,0.716578
24673,Chorraum,"""Im Chorraum der heutigen, im Jahre 1934 gewei...",Chorraum,Architektur: Altarraum in einer Kirche,Raum in einem Chor,0.868111,0.801791,0.833634,0.180003,0.086207,0.542826,0.222222,0.000000,0.222222,0.222222,0.436823
11044,Selm,Selm liegt im Westen Deutschlands,Selm,"eine Stadt in Nordrhein-Westfalen, Deutschland","eine Stadt in Nordrhein-Westfalen, Deutschland",1.000000,1.000000,1.000000,0.974566,0.997685,0.999853,1.000000,1.000000,1.000000,1.000000,1.000000
5893,Felsenpinguin,"""Wie die meisten Pinguinarten nistet der Felse...",Felsenpinguin,Zoologie: eine Art aus der Familie der Pinguine,Zoologie: Vertreter einer Gattung aus der Fami...,0.833297,0.904901,0.867624,0.734324,0.647131,0.644645,0.555556,0.375000,0.555556,0.555556,0.797369
